In [3]:
import pandas as pd 
import numpy as np
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity
import operator
import fastparquet as fp 
import pyarrow as pa 
import pyarrow.parquet as pq

### Modelo de recomendación `Item_Item`

El modelo de recomendación `Item_Item` basado en la similitud de coseno, se utilizara para calcular la similitud entre los datos que representan los elementos a recomendar y el historial o preferencias de un usuario. Cuanto más cercanos estén los datos en términos de similitud de coseno, más parecidos son los elementos y, por lo tanto, más probable es que un usuario que le guste un elemento también le guste otro similar.

importamos el dataset a trabajar 

In [4]:
modelo_item = pd.read_parquet('dataset_limpio/modelo_item.parquet')
modelo_item.head()

,item_id,app_name,genres
0,761140,Lost Summoner Kitty,Action
1,643980,Ironbound,Free to Play
2,670290,Real Pool 3D - Poolians,Casual
3,767400,弹炸人2222,Action
4,772540,Battle Royale Trainer,Action


Procedemos a crear los dummies, para luego crear la matriz necesaria para el analisis de similitud de coseno

In [5]:
#Se crean los dummies 
modelo_item = pd.get_dummies(modelo_item, columns=['genres'], prefix='')

modelo_item = modelo_item.groupby(['item_id', 'app_name']).sum().reset_index()

modelo_item.head()

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Similitud de Coseno

En el contexto de recomendación, los elementos a recomendar se representan como vectores en un espacio multidimensional. Cada elemento está representado por sus características o atributos.
 La similitud del coseno evalúa el ángulo entre dos vectores, variando de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más se considerarán parecidos. Mientras mas parecidos, mejor sera la recomendacion. 

In [8]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])
print(similitudes)

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [9]:
similitudes.shape

(28834, 28834)

Procedemos a crear la funcion `recomendacion_juegos`

In [10]:
def recomendacion_juegos(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = modelo_item[modelo_item['item_id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[modelo_item[modelo_item['item_id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['app_name']
    
    return juegos_recomendados

In [14]:
recomendacion_juegos(772540)

10111                  DW8E: Edit Parts - Equipment Kimono
13405    NARUTO SHIPPUDEN: Ultimate Ninja STORM 4 - Gaa...
13404    NARUTO SHIPPUDEN: Ultimate Ninja STORM 4 - Shi...
13403    Van Helsing Final Cut: High Resolution Texture...
13402                                            Magnetron
Name: app_name, dtype: object

Modelo para Railway

In [15]:
modelo_item

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28829,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28830,2028056,Worms Revolution Season Pass,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28831,2028062,Call of Duty®: Black Ops II Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28832,2028103,Assassin’s Creed® III Season Pass,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dividimos el dataset por la mitad para poder montar el modelo en render

In [28]:
#Cuento filas
cant_filas= len(modelo_item)

#Calculo la mitad
mitad_filas= cant_filas // 10
 
#Selecciono la mitad superior
modelo_railway= modelo_item.iloc[:mitad_filas]

In [29]:
modelo_railway.shape

(2883, 25)

In [31]:
similitudes_railway = cosine_similarity(modelo_railway.iloc[:,3:])

Procedemos a guardar el `modelo_railway` en formato parquet

In [32]:
modelo_railway.to_parquet('dataset_limpio/modelo_railway.parquet')

Procedemos a crear una nueva funcion con el proposito de explorar las similitudes con el juego especificado por parametro. La idea seria facilitar la ejecucion del archivo mediante el proceso de renderizado.

In [33]:
def encontrar_juegos_similares(id):
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_railway.index[modelo_railway['item_id'] == id].tolist()[0]
    
    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_railway.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_railway.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6]
    indices_juegos_similares = indices_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_railway.iloc[indices_juegos_similares]['app_name']
    
    return juegos_similares

In [34]:
encontrar_juegos_similares(3310)

2186    Rocksmith - Gary Clark Jr. - Bright Lights
2659                                 Eets Munchies
2185      Rocksmith - Europe - The Final Countdown
2184         Rocksmith - Whitesnake - Is This Love
Name: app_name, dtype: object

Creamo la funcion `recomendacion_juego`

 Esta función recomienda 5 juegos a partir del juego ingresado


In [35]:
def recomendacion_juego(id: int):
    
    # Verificamos si el juego con game_id existe en df_games
    game = modelo_railway[modelo_railway['item_id'] == id]

    if game.empty:
        return("El juego '{id}' no posee registros.")
    
    # Obtenemos el índice del juego dado
    idx = game.index[0]

    # Tomamos una muestra aleatoria del DataFrame df_games
    sample_size = 2000  # Definimos el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_railway.sample(n=sample_size, random_state=42)  # Ajustamos la semilla aleatoria según sea necesario

    # Calculamos la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_railway.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtenemos las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordenamos los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtenemos los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Listamos los juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [37]:
recomendacion_juego(3310)

{'similar_games': ['Monster Mash',
  'Ticket to Ride',
  'Gumboy Tournament',
  "The Wizard's Pen™",
  'Go Home Dinosaurs!']}